In [63]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup as bs
import re
from openpyxl import Workbook #  결과물을 엑셀 파일로 저장

In [103]:
# url입력
driver = webdriver.Chrome('../../../chromedriver') # 크롬드라이버 경로 설정
url = "https://www.yogiyo.co.kr/" # 사이트 입력
driver.get(url) # 사이트 오픈
driver.maximize_window() # 전체장
time.sleep(2) # 2초 지연

# 검색창 선택
xpath = '''//*[@id="search"]/div/form/input'''  # 검색창
element = driver.find_element_by_xpath(xpath)
element.clear()
time.sleep(2)

# 검색창 입력
value = input()
element.send_keys(value)
time.sleep(2)

# 조회버튼 클릭
search_xpath = '''//*[@id="button_search_address"]/button[2]'''
driver.find_element_by_xpath(search_xpath).click()
time.sleep(3)

try:
    # 검색 콤보상자 선택
    # 선택 : #search > div > form > ul > li:nth-child(3) > a
    search_selector = '#search > div > form > ul > li:nth-child(3) > a'
    search = driver.find_element_by_css_selector(search_selector)
    search.click()
    time.sleep(3)
except:
    pass

경희대 국제


In [67]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") # 스크롤을 가장 아래로 내린다
time.sleep(2)
pre_height = driver.execute_script("return document.body.scrollHeight") # 현재 스크롤 위치 저장
while True :
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")  # 스크롤을 가장 아래로 내린다
    time.sleep(2)
    cur_height = driver.execute_script("return document.body.scrollHeight")  # 현재 스크롤을 저장한다.
    if pre_height == cur_height :
        break
    pre_height = cur_height


time.sleep(3)

# 페이지 소스 출력
html = driver.page_source
soup = bs(html, 'html.parser')

restaurant_list = soup.find_all("div", class_ = "restaurant-name ng-binding") #업체명
restaurant_name = []
for r in restaurant_list:
    restaurant_name.append(r.text)



In [ ]:
# 엑셀 조작
write_wb = Workbook()
write_ws = write_wb.active

In [104]:
# 레스토랑 이름 리스트 순차적으로 검색
start = time.time()
for name in restaurant_name[102:]:
    elem = driver.find_element_by_xpath('//*[@id="category"]/ul/li[1]/a')
    time.sleep(3)
    elem.click()
    time.sleep(2)
    
    elem = driver.find_element_by_xpath("//*[@id='category']/ul/li[15]/form/div/input")
    elem.send_keys(name)
    elem.send_keys(Keys.ENTER)
    time.sleep(2)
    try:
        # 검색결과에서 첫번째 아이템으로 들어감
        elem = driver.find_element_by_xpath('//*[@id="content"]/div/div[5]/div/div/div/div')
        elem.click()
        time.sleep(2)

        # 해당 음식점의 리뷰로 들어감
        try:    
            elem = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a')
            elem.click()

            # 해당 브랜드 리뷰 전체 스크롤 시작
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

            prev_height = driver.execute_script("return document.body.scrollHeight")
            elem = driver.find_element_by_xpath('//*[@id="review"]/li[12]')

            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
                try:
                    elem.click()
                except:
                    pass

                # 페이지 로딩 대기
                time.sleep(2)


                # 현재 문서 높이를 가져와서 저장
                curr_height = driver.execute_script("return document.body.scrollHeight")
                if curr_height == prev_height:
                    break
                prev_height = curr_height
        except:
            pass

        print("리뷰 스크롤 완료")

        # page_source로 스크롤 끝까지 내렸을 때의 html 정보를 가져오게 됨
        soup = bs(driver.page_source, "lxml")
        reviews = soup.find_all("li", attrs={"class": "list-group-item star-point ng-scope"})

        # 만약 리뷰가 한 개도 없다면 다음 브랜드를 찾기 위해 뒤로 가라
        if len(reviews) == 0:
            driver.back()
        for review in reviews:
            taste = review.find("span", attrs={"class": "points ng-binding", "ng-show":"review.rating_taste > 0"})
            quantity = review.find("span", attrs={"class": "points ng-binding", "ng-show":"review.rating_quantity > 0"})
            delivery = review.find("span", attrs={"class": "points ng-binding", "ng-show":"review.rating_delivery > 0"})
            eat_menu = review.find("div", attrs={"class": "order-items default ng-binding", "ng-click": "show_review_menu($event)"})
            com_menu = review.find("p", attrs={"class": "ng-binding", "ng-show": "review.comment"})
            day = review.find("span", attrs = {"ng-bind": "review.time|since", "class": "review-time ng-binding"})
            star = review.find_all("span", attrs = {"class": "full ng-scope"})
            user = review.find("span", attrs={"class": "review-id ng-binding", "ng-show":"review.phone"})

            # <span class="points ng-binding" ng-show="review.rating_taste &gt; 0">5</span> 여기서 5만 가져오려면 get_text() or text
            # try! 이전 자료는 별점이 없기 때문에 리뷰와 주문 메뉴밖에 확인하지 못한다.
            try:
                star_point = int(len(star))
                taste_point = int(taste.text)
                quantity_point = int(quantity.text)
                delivery_point = int(delivery.text)
                menu = eat_menu.text.strip()
                comment = com_menu.text.strip()
                date = day.text.strip()
                user_id = user.text
            except:
                star_point = int(len(star))
                taste_point = 0
                quantity_point = 0
                delivery_point = 0
                menu = eat_menu.text.strip()
                comment = com_menu.text.strip()
                date = day.text.strip()
                user_id = user.get_text()

            write_ws.append([name, user_id, menu, comment, star_point, taste_point, quantity_point, delivery_point, date])
        driver.back()
    except:
        pass
write_wb.save("testxx.xlsx")

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간
#driver.close() # 크롬드라이버 종료

리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료
리뷰 스크롤 완료


In [87]:
name

'소문난장충동왕족발보쌈'

In [95]:
restaurant_name.index('뿅가야채곱창-수원매탄점')

101

In [99]:
restaurant_name[102]

'가본'

In [80]:
for i in range(len(restaurant_name)):
    if restaurant_name[i] == '매운국물방앗간떡볶이-본점':
        print(i)
        break

In [89]:
write_wb.save("test2.xlsx")